# Training Script for EULER CLUSTER

In [1]:
import tensorflow as tf
from time import gmtime, strftime
import datetime
import numpy as np
import math

from attention_model import AttentionModel, set_decode_type
from reinforce_baseline import RolloutBaseline
from train import train_model
from reinforce_baseline import load_tf_model

from generate_data import create_data_on_disk, get_cur_time

# Launch tensorboard via the folowing command in the console:
# tensorboard --logdir logs/EULER --reload_multifile True

# Clear EULER logs:
#!rm -rf src/logs/EULER/*
#!rm -rf src/checkpoints/*

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
#%%prun 
# prun for profiling
tf.config.threading.set_intra_op_parallelism_threads(0)
# Params of model
# ------------------

run_descriptor = "LOCAL_"
GRAPH_SIZE = 5

LEARNING_RATE = 1e-4
START_EPOCH = 0
END_EPOCH = 10000

#hwl = [30,25,60]
hwl = [6,5,12]

BATCH = 100
VAL_BATCH_SIZE = BATCH
ROLLOUT_SAMPLES = BATCH
BATCH_SAVE = 100 # save model every n epochs
run_descriptor = run_descriptor + "_hwl_" + str(hwl[0])+ str(hwl[1])+ str(hwl[2])
# TODO: Vary load level and fragility share

# Only for resuming:
NEW_TRAINING = True # set to false
FILENAME = "LOCAL_ppo_sdvrpish_hwl_6512_2021-08-11-16-26_graph5_batch128_lr0.0001_Clip_0.1_model_checkpoint_epoch_400.h5"

# ------------------


SAMPLES = BATCH
embedding_dim = 128
NUMBER_OF_WP_EPOCHS = 1
GRAD_NORM_CLIPPING = 0.5
VALIDATE_SET_SIZE = VAL_BATCH_SIZE
SEED = 1234

if NEW_TRAINING:
    print("NEW TRAINING")
    FILENAME = '{}_{}_graph{}_batch{}_lr{}_Clip_{}'.format(run_descriptor, strftime("%Y-%m-%d-%H-%M", gmtime()), GRAPH_SIZE, BATCH, LEARNING_RATE, GRAD_NORM_CLIPPING)
    
    #FILENAME = "debug_"
    # Initialize model
    model_tf = AttentionModel(embedding_dim)
    set_decode_type(model_tf, "sampling")
    #print(get_cur_time(), 'model initialized')

    # Create and save validation dataset
    #TODO: currently unused as we generate via seed in train
    """
    validation_dataset = create_data_on_disk(GRAPH_SIZE,
                                             VALIDATE_SET_SIZE,
                                             is_save=True,
                                             filename=FILENAME,
                                             is_return=True,
                                             seed = SEED)
    """
    validation_dataset = 0
    #print(get_cur_time(), 'validation dataset created and saved on the disk')

    # Initialize optimizer
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=LEARNING_RATE,
        decay_steps=10000,
        decay_rate=0.9)
    #optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=GRAD_NORM_CLIPPING, epsilon=1e-8)

    # Initialize baseline
    baseline = RolloutBaseline(model_tf,
                               wp_n_epochs = NUMBER_OF_WP_EPOCHS,
                               epoch = 0,
                               num_samples=ROLLOUT_SAMPLES,
                               filename = FILENAME,
                               from_checkpoint = False,
                               embedding_dim=embedding_dim,
                               graph_size=GRAPH_SIZE
                               )
    #print(get_cur_time(), 'baseline initialized')

    model_t = train_model(optimizer,
                model_tf,
                baseline,
                validation_dataset,
                samples = SAMPLES,
                batch = BATCH,
                val_batch_size = VAL_BATCH_SIZE,
                start_epoch = START_EPOCH,
                end_epoch = END_EPOCH,
                from_checkpoint = False,
                grad_norm_clipping = GRAD_NORM_CLIPPING,
                batch_save = BATCH_SAVE,
                graph_size = GRAPH_SIZE,
                filename = FILENAME,
                hwl = hwl
                )
else:
    print("RESUMING TRAINING")
    
    folder = "src/checkpoints/"
    MODEL_PATH = folder+FILENAME
    BASELINE_MODEL_PATH = folder+'baseline_checkpoint.h5'
    #VAL_SET_PATH = 'Not needed'

    # Initialize model
    model_tf = load_tf_model(MODEL_PATH,
                             embedding_dim=embedding_dim,
                             graph_size=GRAPH_SIZE)
    set_decode_type(model_tf, "sampling")
    #print(get_cur_time(), 'model loaded')

    # Create and save validation dataset
    validation_dataset = 0 # read_from_pickle(VAL_SET_PATH)
    #print(get_cur_time(), 'validation dataset loaded')

    # Initialize optimizer
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=LEARNING_RATE,
        decay_steps=1000,
        decay_rate=0.9)
    #optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=GRAD_NORM_CLIPPING, epsilon=1e-8)

    # Initialize baseline
    baseline = RolloutBaseline(model_tf,
                               wp_n_epochs = NUMBER_OF_WP_EPOCHS,
                               epoch = START_EPOCH,
                               num_samples=ROLLOUT_SAMPLES,
                               filename = FILENAME,
                               from_checkpoint = True,
                               embedding_dim=embedding_dim,
                               graph_size=GRAPH_SIZE,
                               path_to_checkpoint = BASELINE_MODEL_PATH)
    print(get_cur_time(), 'baseline initialized')

    train_model(optimizer,
                model_tf,
                baseline,
                validation_dataset,
                samples = SAMPLES,
                batch = BATCH,
                val_batch_size = VAL_BATCH_SIZE,
                start_epoch = START_EPOCH,
                end_epoch = END_EPOCH,
                from_checkpoint = True,
                grad_norm_clipping = GRAD_NORM_CLIPPING,
                batch_save = BATCH_SAVE,
                graph_size = GRAPH_SIZE,
                filename = FILENAME
                )

NEW TRAINING
Generating Env with seed  0
Evaluating baseline model on baseline dataset (epoch = 0)
Current decode for baseline is type: greedy
Generating Env with seed  1234
Generating Env with seed  0

 ########## 
 Starting Epoch 0
Current decode for training is type: sampling
Generating Env with seed  0
Benchmark
Current decode for baseline is type: greedy
2021-09-12 00:41:18 Epoch 0: Loss: -0.4000000059604645: Cost: 1.0 Time: 9.5sec, Remaining: 26.39h
Generating Env with seed  0
Evaluating candidate model on baseline dataset (callback epoch = 0)
Current decode for baseline is type: greedy
Epoch 0 candidate mean 0.8215309977531433, baseline epoch 0 mean 1.0330899953842163, difference -0.211558997631073
p-value: 0.00012292845785577392
Update baseline
Generating Env with seed  0
Evaluating baseline model on baseline dataset (epoch = 0)
Current decode for baseline is type: greedy
alpha was updated to 1.0
Generating Env with seed  1

 ########## 
 Starting Epoch 1
Current decode for bas

KeyboardInterrupt: 

In [ ]:
# Uncomment to be linked to the Tensorboard port to view results
#!tensorboard --logdir src/src/logs/EULER --reload_multifile True

In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir logs/debug_stuff

In [ ]:
#env.make_gif(batch=0, my_title="gend", fps = 3)

In [ ]:
np.random.randint(5,8)